In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd

test = pd.read_pickle('../results/Somite.pkl')
test

X:153066000-153070096     [0.016203300704057205, 0.016203300704057205, 0...
14:105454000-105458096    [0.440591878718832, 0.4400747521006175, 0.4395...
10:79848000-79852096      [0.09515129775148486, 0.0968750531455335, 0.09...
7:158174000-158178096     [0.3611267550531898, 0.35992012627735576, 0.35...
11:6454000-6458096        [0.9330687947985281, 0.9325516681803134, 0.932...
                                                ...                        
1:122534000-122538096     [0.05205741290026889, 0.052229788439673755, 0....
4:87346000-87350096       [0.24546276811252615, 0.24529039257312127, 0.2...
1:116528000-116532096     [0.3307886601179338, 0.3319952888937678, 0.333...
11:71156000-71160096      [0.33440854644543594, 0.33509804860305537, 0.3...
10:119150000-119154096    [0.6298602209853725, 0.6339972339310893, 0.638...
Length: 289162, dtype: object

In [2]:
test.memory_usage()

13080776

In [4]:
test = pd.DataFrame(test.tolist(), index= test.index)

In [ ]:
import numpy as np

test = test.apply(pd.Series)
test

In [5]:
test

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024
X:153066000-153070096,0.016203,0.016203,0.016203,0.016203,0.016203,0.016203,0.016203,0.016203,0.016203,0.016203,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
14:105454000-105458096,0.440592,0.440075,0.439558,0.439558,0.439385,0.439213,0.439040,0.438868,0.438696,0.438351,...,0.037750,0.033613,0.029476,0.025339,0.021030,0.016720,0.012411,0.008274,0.004137,0.0
10:79848000-79852096,0.095151,0.096875,0.098771,0.100667,0.102563,0.104460,0.106356,0.108252,0.110148,0.112044,...,0.021030,0.018617,0.016203,0.013790,0.011377,0.008964,0.006723,0.004482,0.002241,0.0
7:158174000-158178096,0.361127,0.359920,0.358541,0.357162,0.355783,0.354404,0.353025,0.351646,0.350439,0.349233,...,0.020513,0.018099,0.015859,0.013618,0.011377,0.009136,0.006895,0.004654,0.002413,0.0
11:6454000-6458096,0.933069,0.932552,0.932035,0.931690,0.931690,0.931862,0.932207,0.932552,0.932896,0.933414,...,0.058435,0.052747,0.047059,0.040336,0.033613,0.026891,0.020168,0.013445,0.006723,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1:122534000-122538096,0.052057,0.052230,0.052402,0.052575,0.052747,0.052919,0.053092,0.053264,0.053436,0.053609,...,0.006206,0.005516,0.004827,0.004137,0.003448,0.002758,0.002069,0.001379,0.000690,0.0
4:87346000-87350096,0.245463,0.245290,0.245118,0.245118,0.245118,0.245290,0.245290,0.245118,0.244946,0.244773,...,0.006206,0.005516,0.004827,0.004137,0.003448,0.002758,0.002069,0.001379,0.000690,0.0
1:116528000-116532096,0.330789,0.331995,0.333202,0.334409,0.335615,0.336822,0.338201,0.339752,0.341304,0.342855,...,0.022926,0.020513,0.018099,0.015514,0.012928,0.010343,0.007757,0.005171,0.002586,0.0
11:71156000-71160096,0.334409,0.335098,0.335788,0.336477,0.337167,0.337856,0.338546,0.338890,0.339407,0.340269,...,0.015514,0.013790,0.012066,0.010343,0.008619,0.006895,0.005171,0.003448,0.001724,0.0


In [7]:
test.memory_usage()

Index    10767480
0         2313296
1         2313296
2         2313296
3         2313296
           ...   
1020      2313296
1021      2313296
1022      2313296
1023      2313296
1024      2313296
Length: 1026, dtype: int64

In [ ]:
ATAC = pd.concat(pd.read_pickle(f) for f in ['../results/ATAC/D8/Somite.pkl', '../results/ATAC/D22-15/Myoblast.pkl'])


In [ ]:
import pandas as pd
import pickle

#Create test dataset
ATAC = pd.concat(pd.read_pickle(f) for f in ['../results/Somite.pkl', '../results/Myoblast.pkl'])
with open('../results/ATAC_peaks.pkl', 'wb') as file:
            pickle.dump(ATAC, file)

In [3]:
from pytorch_datasets import PeaksDataset
from torch.utils.data import DataLoader

data_dir = '../results/'
chrom_train = ['1','2','3','4','5','7','8','9','10','11','12','14','15','16','17','18','19','20','21','X','Y']
chrom_test = ['6']

nb_back = 100
pseudo_bulk_order = ["D8Somite","D22-15Myoblast"]

train_dataset = PeaksDataset(data_dir + 'peaks_seq.pkl', data_dir + 'background_GC_matched.pkl',
                                 data_dir + 'ATAC_peakst.pkl', data_dir + 'ATAC_backgroundt.pkl', 
                                 chrom_test, pseudo_bulk_order, nb_back)
train_dataloader = DataLoader(train_dataset, batch_size=32,
                        shuffle=True, num_workers=2)

In [3]:
import tqdm
for i, data in tqdm(enumerate(train_dataloader)):
    inputs, tracks = data 
    print(inputs)
    
    break

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10dc99fd0>>
Traceback (most recent call last):
  File "/Applications/anaconda3/envs/LeKIra/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [7]:
import torch
from models import BPNet

m = BPNet()
input = torch.randint(low=0, high=2, size=(32, 4, 4096)).float()
tracks = torch.abs(torch.randn(size=(32,1025)))
print(input.shape)
x, profile, count = m(input)

profile[10].shape

torch.Size([32, 4, 4096])


torch.Size([1024])

In [48]:
from eval_metrics import ATACloss, profile_metrics

criterion = ATACloss(weight_MSE=1)
loss = criterion(tracks, profile, count)
loss

tensor(677285.0625, grad_fn=<AddBackward0>)

In [65]:
profile_metrics(tracks, profile)


[1.0242868233369062,
 1.0269171755998823,
 1.0252296955673368,
 1.0323565062471467,
 1.0281575491595099,
 1.0207284133491756,
 1.0175975697749555,
 1.0176447461398825,
 1.035496942094676,
 1.0181153806221328,
 1.0151624622692277,
 1.025698947062699,
 1.0315933184393649,
 1.0168293074189176,
 1.0231442962397,
 1.015189499130307,
 1.0083193528481231,
 1.0168074147803308,
 1.0283298762745399,
 1.012215962803128,
 1.022288898699505,
 1.0262645964559594,
 1.0261720716508131,
 1.0285287157616725,
 1.018175952054657,
 1.0145428424613363,
 1.0245412981016622,
 1.0221562695444164,
 1.0237888410379068,
 1.0275504210794684,
 1.02138185085399,
 1.0190031268863184]

In [17]:
import torch
from models import CATAC

m = CATAC(nb_pred=15, nb_cell_type_CN=4, nb_conv=8)
input = torch.randn(32, 4, 4096)
print(input.shape)
x, profile, count = m(input)

profile[10].shape

torch.Size([32, 4, 4096])


torch.Size([32, 1024])

BPNet uses a composite loss function consisting of a linear combination of a mean squared error (MSE) loss on the log of the total counts and a multinomial negative log likelihood loss (MNLL) for the profile probability output. We use a weight of [4.9, 4.3, 18.5, 9.8, 8.9, 4.8, 4.6, 4.9, 12.4, 15.4, 4.3, 6.3, 1.4, 2.6, 7.6, 2.3, 16.3, 7.1 & 3.7] for the MSE loss for clusters c0–c20 (c15-c16 combined as one model), and a weight of 1 for the MNLL loss in the linear combination. The MSE loss weight is derived as the median of total counts across all peak regions for each cluster divided by a factor of 10 

In [10]:
pip install deeplift

  Preparing metadata (setup.py) ... done
  Created wheel for deeplift: filename=deeplift-0.6.13.0-py3-none-any.whl size=36424 sha256=091fc6dcfd9d22cc479a2b139f6743120df92c7fe99c9b5127633c6a6a2ad196
  Stored in directory: /Users/adorman/Library/Caches/pip/wheels/15/47/9c/66fd18175f190aa68311228c694ebc713548ae719974f1e25b
Successfully built deeplift
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install shap

In [13]:
from deeplift.dinuc_shuffle import dinuc_shuffle
import torch
import pandas as pd
from data_processing.utils_data_preprocessing import one_hot_encode

import shap

def compute_shap_score(model,seq,background):
        explainer = shap.DeepExplainer(
            (model, model.linear), background)

        raw_scores = explainer.shap_values(s, background[i], progress_message=100)
    
        for i,s in enumerate(seq):

def compute_importance_score(path_model, path_sequence, ):

    #Load the model and sequenece to predict
    model = torch.load(path_model)
    seq = pd.read_pickle(path_sequence).sequenece

    #On-hot encode the sequences
    seq = seq.apply(lambda x: one_hot_encode(x))
    
    #Create shuffled sequences for background
    background = [dinuc_shuffle(s, num_shufs=20) for s in seq]

    #Compute importance score for each base of sequences
    

In [19]:
seq = pd.read_pickle('../results/background_GC_matched.pkl').sequence.iloc[:100]
seq = seq.apply(lambda x: one_hot_encode(x))

background = [dinuc_shuffle(s, num_shufs=20) for s in seq]
